## Nhận diện chữ số trong tập dữ liệu MNIST dùng CNN sử dụng thư viện Keras

In [1]:
import numpy as np
# Linear stack of layers, feed-forwad CNN (dễ dàng kiểm soát các layer trong CNN)
from keras.models import Sequential

Using TensorFlow backend.


### Các "core" layer thường được sử dụng trong hầu hết các neural network model:
* Dense: được sử dụng để thay đổi số chiều của vector. Implement: output = activation(dot(input, kernel) + bias); kernel: weights matrix -> tạo ra n-dimension output. Nếu 
* Dropout: được dùng để regularized, tránh overfitting bằng cách đặt ngẫu nhiên 1 phần(0->1) của input vector thành 0 tại mỗi lần cập nhật trong thời gian đào tạo
* Flatten: làm phẳng input(chuyển input_shape -> 1-dimension input), không ảnh hưởng đến batch size

In [2]:
from keras.layers import Dense, Dropout, Flatten

### Một số layer khác:
* Convolution 2D: Layer này tạo ra convolution kernel được convolved với input layer để tạo ra một tensor của output. Nếu activation is not None nó sẽ được áp dụng cho output. Có thể set 'chanel_first' hoặc 'chanel_last'(default)
* Max pooling 2D: Đi qua input và giữ lại max của mỗi pool, giá trị pool_size là kích thước cho mỗi pool, vd pool_size=(2,2)

In [3]:
from keras.layers import Conv2D, MaxPooling2D

In [4]:
from keras.datasets import mnist
from keras.utils import np_utils
from keras import backend as K
import keras
batch_size = 128
num_classes = 10 # tương ứng với 10 chữ số
epochs = 12
# input image dimensions
img_rows, img_cols = 28, 28
# chia train data and test data
(x_train, y_train), (x_test, y_test) = mnist.load_data()
# chuyển đổi tập dữ liệu từ (n, width, height) sang (n, depth, width, height) nếu 'channels_first'
# từ (n, width, height) sang (n, width, height, depth) nếu 'channels_last'
if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)
# Bước tiền xử lý cuối cùng cho input là chuyển thành float32 
# và chuẩn hóa các giá trị dữ liệu trong đoạn [0, 1]
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [5]:
# Chuyển đổi mảng 1 chiều thành ma trận lớp 10 chiều
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
y_train.shape

(60000, 10)

In [6]:
# Khai báo mô hình Sequential
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), 
                 activation='relu', 
                 input_shape=input_shape))
# tham số đầu tiên đại diện cho số lượng các convolution filter được sử dụng,
# kích thước mỗi bước là (1,1) theo mặc định và có thể được điều chỉnh bằng tham số 'subsample'.
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
# Phải làm phẳng trước khi dùng Dense nếu số chiều của input >= 2
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))
# Layer cuối có output size là 10, tương ứng với 10 class của chữ số.
# Configures the model for training: khai báo loss function và optimizer
# tham số metrics để đánh giá mô hình trong quá trình train và test, thường dùng ['accuracy']
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])
# Huấn luyện model với số epoch được chỉ định
# chế độ verbose: 0 = silent, 1 = verbose, 2 = one log line per epoch.
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
# Return loss value & metrics values in test mode
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 60000 samples, validate on 10000 samples
Epoch 1/12
60000/60000 [==============================] - 266s - loss: 0.3500 - acc: 0.8912 - val_loss: 0.0802 - val_acc: 0.9747
Epoch 2/12
60000/60000 [==============================] - 276s - loss: 0.1185 - acc: 0.9651 - val_loss: 0.0534 - val_acc: 0.9830
Epoch 3/12
60000/60000 [==============================] - 279s - loss: 0.0900 - acc: 0.9730 - val_loss: 0.0448 - val_acc: 0.9850
Epoch 4/12
60000/60000 [==============================] - 279s - loss: 0.0757 - acc: 0.9780 - val_loss: 0.0399 - val_acc: 0.9876
Epoch 5/12
60000/60000 [==============================] - 280s - loss: 0.0635 - acc: 0.9812 - val_loss: 0.0371 - val_acc: 0.9882
Epoch 6/12
60000/60000 [==============================] - 293s - loss: 0.0582 - acc: 0.9830 - val_loss: 0.0322 - val_acc: 0.9889
Epoch 7/12
60000/60000 [==============================] - 278s - loss: 0.0543 - acc: 0.9840 - val_loss: 0.0309 - val_acc: 0.9895
Epoch 8/12
60000/60000 [=======================